In [6]:
# Testing multiple sixth_reg based models

In [7]:
%run project_base.ipynb

In [8]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [9]:
class BilinearInterp(FineGridPredictorMixin, BilinearInterpBase):
    pass
    
    
class NearestInterp(FineGridPredictorMixin, NearestInterpBase):
    pass
        
    
class BilinearDiffNearest(_PredictorBase):
    @classmethod
    def predict_single(cls, Xi, y):
        bilinear = BilinearInterp.predict_single(Xi, y)
        nearest = NearestInterp.predict_single(Xi, y)
        return [b - n for b, n in zip(bilinear, nearest)]
            

def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the feature is the solution on a coarse grid and the label is the 
    difference between solution on a fine grid and the prediction given by
    bilinear interpolation.
    """
    
    # Random solution to the CH equation
    c = np.random.uniform(0, 10)
    peakon = Peakon(c=c)
    # Random location
    t = np.random.uniform(0, 40)
    x = c * t + np.random.uniform(-5, 5)
    
    # Grids at the location
    cg = coarse_grid((t, x))
    fg = fine_grid((t, x))
    # Features: the solution on the coarse grid
    X = peakon.on_grid(cg)
    # Labels: the difference between the solution on the fine grid and the
    # prediction given by bilinear interpolation
    y = peakon.on_grid(fg)
    # Predict expects to be given a batch of elements, so we have to wrap
    # the input X, y in a list, and then extract the prediction from a list
    # of one element.
    predictor = NearestInterp.predict(input_fn=lambda: ({'X': [X]}, [y]))
    y_prediction = next(predictor)[0]
    
    return X, y - y_prediction

In [11]:
### DNN hyperparameters
hidden_units = [2000] * 20
# The number of fine grid points.
logits = (fine_grid_fineness.t + 1) * (fine_grid_fineness.x + 1)
drop_rate = 0.4

### Training hyperparameters #1
gradient_clip = 5.0  # May be set to None to turn off gradient clipping

### Logging/saving hyperparameters
log_steps = 100  # How many steps to print the current loss.

for uuid in range(14, 100):
    model_dir = './saved_models/sixth_reg/'
    model_dir = model_dir_str(model_dir, hidden_units, logits, drop_rate, uuid=uuid)

    dnn = Sequential.create_dnn(hidden_units, logits, drop_rate=drop_rate,
                                model_dir=model_dir, log_steps=log_steps,
                                gradient_clip=gradient_clip)

    if TRAIN:
        print('-----uuid: {} ------'.format(uuid))
        ### Training hyperparameters #2
        batch_size = 30
        steps = 500
        train_input_fn = BatchData(gen_one_data, batch_size)
        dnn.train(input_fn=train_input_fn, max_steps=steps)
        print('----------------------')

INFO:tensorflow:Using config: {'_model_dir': './saved_models/sixth_reg/2000x20_121_D04_UUID14', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f35fc62afd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
-----uuid: 14 ------
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/sixth_reg/2000x20_121_D04_UUID14/model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running l

KeyboardInterrupt: 